## Exploratory Data Analysis(Zillow Dataframe)
### Corey Solitaire
#### 10.13. 2020

In [1]:
import acquire
import prepare
import wrangle_zillow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df, X_train_explore, X_train_scaled, X_validate_scaled, X_test_scaled = wrangle_zillow.wrangle_zillow(wrangle_zillow.get_zillow_data(cached=False)) 


In [4]:
X_train_explore.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,latitude,longitude,lotsizesquarefeet,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,tdate,heatingorsystemdesc,propertylandusedesc,county
11612,11600673,3.0,3.0,7.0,1945.0,1945.0,3.0,34047894.0,-118376073.0,6322.0,...,784417.0,2016.0,548032.0,9564.10,6.037217e+13,0.001199,52,2,10,0
8246,13069625,3.0,4.0,8.0,2484.0,2484.0,3.0,34034760.0,-117827592.0,9595.0,...,575097.0,2016.0,254536.0,6942.29,6.037403e+13,-0.036281,37,1,10,0
30920,17072012,4.5,4.0,8.0,3436.0,3436.0,4.0,34281197.0,-119228656.0,9767.0,...,560991.0,2016.0,197991.0,5943.90,6.037137e+13,-0.030303,112,6,10,2
3593,10767620,3.0,3.0,8.0,1972.0,1972.0,3.0,34212494.0,-118570382.0,5373.0,...,246702.0,2016.0,107778.0,3164.96,6.037134e+13,0.002852,17,1,10,0
50544,11186544,3.0,4.0,7.0,2363.0,2363.0,3.0,34622804.0,-118200741.0,6007.0,...,105757.0,2016.0,16542.0,2598.21,6.037910e+13,-0.027416,171,1,10,0
